In [640]:
import pandas as pd 
import numpy as np 

In [641]:
# getting NamUs Missing Persons database in 3 parts 
db1_missing = pd.read_csv("data/missing-namus-first-pull.csv")
db2_missing = pd.read_csv("data/missing-namus-second-pull.csv") 
db3_missing = pd.read_csv("data/missing-namus-third-pull.csv") 

# combining all 3 into one dataframe 
missing_df = pd.concat([db1_missing, db2_missing, db3_missing], ignore_index=True, sort=False)

In [642]:
missing_df.columns

Index(['Case Number', 'DLC', 'Legal Last Name', 'Legal First Name',
       'Missing Age', 'City', 'County', 'State', 'Biological Sex',
       'Race / Ethnicity', 'Date Modified'],
      dtype='object')

In [643]:
# in case we need to drop an idx
drop_row_idx = []

# purpose: make every age value into a numerical value 
def ageToNumerical(df): 
    for idx, i in enumerate(df["Missing Age"]): 
        # for null/empty/nan values
        if pd.isna(i) or i == "" or isinstance(i, (int, float)) or pd.isnull(i): 
            drop_row_idx.append(idx) 

    # handling special cases 
        # 1 - 10 age 
        elif str(i).strip() == '< 1 Year' or str(i).strip() == '1 - 2 Years': 
            df.loc[idx, "Missing Age"] = 1

        elif str(i).strip() == '3 - 4 Years': 
            df.loc[idx, "Missing Age"] = 3

        # 10 - 20 age 
        elif str(i).strip() == '14 - 15 Years': 
            df.loc[idx, "Missing Age"] = 14

        elif str(i).strip() == '15 - 17 Years': 
            df.loc[idx, "Missing Age"] = 16

        elif str(i).strip() == '16 - 18 Years' or str(i).strip() == '16 - 17 Years': 
            df.loc[idx, "Missing Age"] = 17

        elif str(i).strip() == '18 - 20 Years': 
            df.loc[idx, "Missing Age"] = 19


        # 20 - 30 age
        elif str(i).strip() == '21 - 22 Years': 
            df.loc[idx, "Missing Age"] = 21

        elif str(i).strip() == '23 - 24 Years' or str(i).strip() == '23 - 25 Years': 
            df.loc[idx, "Missing Age"] = 23

        elif str(i).strip() == '29 - 31 Years' or str(i).strip() == '25 - 35 Years' or str(i).strip() == '25 - 39 Years': 
            df.loc[idx, "Missing Age"] = 30

        # 30 - 40 age 
        elif str(i).strip() == '30 - 40 Years' or str(i).strip() == '35 - 37 Years' or str(i).strip() == '36 - 37 Years' or str(i).strip() == '33 - 34 Years' or str(i).strip() == '34 - 35 Years': 
            df.loc[idx, "Missing Age"] = 35

        elif str(i).strip() == '37 - 38 Years' or str(i).strip() == '30 - 31 Years': 
            df.loc[idx, "Missing Age"] = 37

        elif str(i).strip() == '39 - 41 Years' or str(i).strip() == '40 - 41 Years': 
            df.loc[idx, "Missing Age"] = 40

        # 40 - 50 age 
        elif str(i).strip() == '42 - 44 Years' or str(i).strip() == '40 - 47 Years' or str(i).strip() == '40 - 42 Years' or str(i).strip() == '44 - 45 Years' or str(i).strip() == '43 - 44 Years': 
            df.loc[idx, "Missing Age"] = 43

        elif str(i).strip() == '49 - 50 Years': 
            df.loc[idx, "Missing Age"] = 50

        # 50 - 60 age 
        elif str(i).strip() == '50 - 60 Years' or str(i).strip() == '51 - 52 Years': 
            df.loc[idx, "Missing Age"] = 55

        elif str(i).strip() == '55 - 65 Years' or str(i).strip() == '59 - 60 Years': 
            df.loc[idx, "Missing Age"] = 60

        elif str(i).strip() == '56 - 60 Years': 
            df.loc[idx, "Missing Age"] = 58

        # senior 
        elif str(i).strip() == '65 - 71 Years' or str(i).strip() == '80 - 82 Years': 
            df.loc[idx, "Missing Age"] = 70

    # most common cases
        # ex. "20 Years" 
        else: 
            df.loc[idx, "Missing Age"] = i[0:-5]

    # string to float 
    df["Missing Age"] = df["Missing Age"].astype(float)

    return df 
    

missing_df_1 = ageToNumerical(missing_df) 

In [644]:
# drop nan/null columns 
print(drop_row_idx)
missing_df_1 = missing_df_1.drop(index=drop_row_idx)

[3191, 3470, 5003, 5504, 9450]


In [645]:
missing_df_1["Missing Age"].describe()

count    25055.000000
mean        34.406546
std         17.279730
min          1.000000
25%         21.000000
50%         32.000000
75%         45.000000
max        116.000000
Name: Missing Age, dtype: float64

In [646]:
missing_df_1["Missing Age"].unique()

array([ 16.,  18.,  35.,  78.,  15.,  27.,  79.,  17.,  14.,  32.,  54.,
        80.,  64.,  13.,  73.,  39.,  66.,  23.,  29.,  68.,  83.,  25.,
        24.,  38.,  31.,  37.,  20.,  49.,  12.,  69.,  61.,  26.,  63.,
        53.,  67.,  72.,  46.,   7.,  44.,  11.,  36.,  52.,  60.,  86.,
        40.,  51.,  21.,  56.,  43.,  41.,  34.,  50.,  62.,  45.,  57.,
        42.,  19.,  74.,  33.,   1.,  59.,  70.,   8.,  71.,  48.,  77.,
        28.,  22.,   5.,   6.,  47.,  30.,  58.,  65.,  55.,   4.,  10.,
         2.,  92.,  75.,  84.,  76.,   9.,   3.,  81.,  82.,  85.,  87.,
        90.,  89.,  93.,  88.,  91.,  94.,  98.,  95.,  99., 114.,  96.,
       116.])

In [647]:
# in case we need to drop an idx
drop_row_idx = []

# purpose: turning int values into age categories 
def ageNumericalToCategory(df): 
    # new column for age categories 
    df["Age Category"] = ""

    for idx, i in enumerate(df["Missing Age"]): 
        # ages 0 - 2
        if i < 3: 
            df.loc[idx, "Age Category"] = "Infant/Toddler (0-2)"
        # ages 3 - 12
        elif i < 13: 
            df.loc[idx, "Age Category"] = "Children (3-12)"
        # ages 13 - 17
        elif i < 18: 
            df.loc[idx, "Age Category"] = "Teen/Adolescent (13-17)"
        # ages 18 - 24
        elif i < 25: 
            df.loc[idx, "Age Category"] = "Young Adult (18-24)" 
        # ages 25 - 44
        elif i < 45: 
            df.loc[idx, "Age Category"] = "Adult (25-44)"
        # ages 45 - 64
        elif i < 65: 
            df.loc[idx, "Age Category"] = "Middle Age (45-64)" 
        # ages 65 and up 
        elif i >= 65: 
            df.loc[idx, "Age Category"] = "Senior (65+)" 
        else: 
            df.loc[idx, "Age Category"] = "Adult (25-44)"

    # drop old age column 
    df = df.drop(["Missing Age"], axis=1)
    # drop nan/null columns 
    df.drop(drop_row_idx, inplace=True)

    return df 

missing_df_2 = ageNumericalToCategory(missing_df_1)     

In [648]:
# Age category unique values 
missing_df_2["Age Category"].unique()

array(['Teen/Adolescent (13-17)', 'Young Adult (18-24)', 'Adult (25-44)',
       'Senior (65+)', 'Middle Age (45-64)', 'Children (3-12)',
       'Infant/Toddler (0-2)', ''], dtype=object)

In [649]:
# length of missing person database 
len(missing_df_2)

25060

In [650]:
missing_df_2["Race / Ethnicity"].value_counts().head(10)

Race / Ethnicity
White / Caucasian                                     14071
Black / African American                               4084
Hispanic / Latino                                      3465
White / Caucasian, Hispanic / Latino                   1141
American Indian / Alaska Native                         797
Asian                                                   577
Other                                                   173
Hispanic / Latino, White / Caucasian                    125
Uncertain                                                92
White / Caucasian, American Indian / Alaska Native       40
Name: count, dtype: int64

In [651]:
# purpose: reduce Race/Ethnicity categories 
def reduceRaceCategories(df): 
    # new column for new race categories
    df["Race Category"] = ""

    for idx, i in enumerate(df["Race / Ethnicity"]): 
        # to new column for race category 
        if i == "Hispanic / Latino": 
            df.loc[idx, "Race Category"] = "Hispanic / Latino" 

        elif i == "White / Caucasian": 
            df.loc[idx, "Race Category"] = "White / Caucasian" 
            
        elif i == "Black / African American": 
            df.loc[idx, "Race Category"] = "Black / African American" 
            
        elif i == "American Indian / Alaska Native": 
            df.loc[idx, "Race Category"] = "American Indian / Alaska Native" 
            
        elif i == "Asian": 
            df.loc[idx, "Race Category"] = "Asian"
        # for mixed races 
        else: 
            df.loc[idx, "Race Category"] = "Mix / Other / Unknown"

    df = df.drop(["Race / Ethnicity"], axis=1)

    return df 

missing_df_3 = reduceRaceCategories(missing_df_2)  

In [652]:
missing_df_3["Race Category"].unique()

array(['White / Caucasian', 'Mix / Other / Unknown',
       'Black / African American', 'Hispanic / Latino',
       'American Indian / Alaska Native', 'Asian'], dtype=object)

In [653]:
missing_df_3.head()

,Case Number,DLC,Legal Last Name,Legal First Name,City,County,State,Biological Sex,Date Modified,Age Category,Race Category
0,MP128712,09/04/2024,Campirano,Nixon,Goodyear,Maricopa,AZ,Male,09/06/2024,Teen/Adolescent (13-17),White / Caucasian
1,MP128606,08/31/2024,Batra,Anjali,Morgan Hill,Santa Clara,CA,Female,09/04/2024,Young Adult (18-24),Mix / Other / Unknown
2,MP128466,08/30/2024,CLARK,JOSHUA,Mulberry,Polk,FL,Male,09/06/2024,Adult (25-44),White / Caucasian
3,MP128454,08/30/2024,Turney-Long,Alinda,Winter Haven,Polk,FL,Female,09/06/2024,Teen/Adolescent (13-17),White / Caucasian
4,MP128473,08/30/2024,COTE,KATHY,West Palm Beach,Palm Beach,FL,Female,09/06/2024,Senior (65+),White / Caucasian


In [654]:
# checking for non calendar values 
for idx, i in enumerate(missing_df_3["DLC"]): 
    if "/" not in str(i): 
        # print(missing_df_3.iloc[[idx]])
        # print("----------------------")
        print(i)

# removing none calendar values 
missing_df_3 = missing_df_3.dropna(subset=["DLC"])
        

nan
nan
nan
nan
nan


In [655]:
# check for non calendar values again 
for i in missing_df_3["DLC"]: 
    if "/" not in str(i): 
        print(i)

In [656]:
# purpose: get month value 
def getMonth(df): 
    # new month column 
    df["Month"] = ""

    for idx, i in enumerate(df["DLC"]): 
        i = i.split("/")[0]

        if i == "01": 
            df.loc[idx, "Month"] = "Jan."

        elif i == "02": 
            df.loc[idx, "Month"] = "Feb."

        elif i == "03": 
            df.loc[idx, "Month"] = "Mar."

        elif i == "04": 
            df.loc[idx, "Month"] = "Apr."

        elif i == "05": 
            df.loc[idx, "Month"] = "May."

        elif i == "06": 
            df.loc[idx, "Month"] = "Jun."

        elif i == "07": 
            df.loc[idx, "Month"] = "Jul."

        elif i == "08": 
            df.loc[idx, "Month"] = "Aug."

        elif i == "09": 
            df.loc[idx, "Month"] = "Sep."

        elif i == "10": 
            df.loc[idx, "Month"] = "Oct."

        elif i == "11": 
            df.loc[idx, "Month"] = "Nov."

        elif i == "12": 
            df.loc[idx, "Month"] = "Dec."

    # removing nan values 
    df = df.dropna(subset=["Month"])

    return df 

missing_df_4 = getMonth(missing_df_3)

In [657]:
missing_df_4["Month"].unique()

array(['Sep.', 'Aug.', 'Jul.', 'Jun.', 'May.', 'Apr.', 'Mar.', 'Feb.',
       'Jan.', 'Dec.', 'Nov.', 'Oct.', ''], dtype=object)

In [659]:
missing_df.head()

from IPython.display import FileLink, FileLinks

missing_df_3.to_csv('./updated-missing-person-data.csv', index=False)

FileLinks('./updated-missing-person-data.csv')

ValueError: Cannot display a file using FileLinks. Use FileLink to display './updated-missing-person-data.csv'.

In [660]:
missing_df_4["Biological Sex"].value_counts()

Biological Sex
Male      15702
Female     9347
Other         6
Name: count, dtype: int64